In [1]:
## Instalaciones

%pip install torch
%pip install open3d
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
## Dependencias

from typing import List
import torch
import os
import open3d as o3d
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
from model import PointNetClassifier, PointNetLoss, PointNetKAN, TTAClassifier
from modelnet10 import ModelNetClass, ModelNet, DatasetType
from utils.csv import save_loss_dict
from utils.transformation import (Normalization,
                                  Rotation, Translation, Reflection, Scale,
                                  DropRandom, DropSphere, Jittering, Noise)
from trainer import PointNetTrainer


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}.")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Using cuda.


In [3]:
# parámetros globales
checkpoint_freq = 25

# parámetros del dataset
classes = [label for label in ModelNetClass]
batch_size = 32
dim = 3
num_points = 1024
num_classes = len(classes)

# hiperparámetros
num_global_feats = 1024     # número de features globales calculadas
learning_rate = 0.001
reg_weight = 0.001
gamma = 2                   # Recomendado por el paper de focal loss

# TODO: Más adelante usar alpha para clases imbalanceadas

In [4]:

# dataset de entrenamiento
t = [Rotation(), Reflection(), Scale(max_ratio=2.5),
    Jittering(max_units=0.005), DropRandom(loss_ratio=0.4), Noise()]

train_data = ModelNet(classes, DatasetType.TRAIN, repetitions=3, transformations=t, preserve_original=False)
validation_data = ModelNet(classes, DatasetType.VALIDATION, repetitions=3, transformations=t, preserve_original=False)

In [5]:
# Función de entrenamiento
def train(
        epochs: int,
        name: str,
        num_global_feats: int,
        learning_rate: int,
        use_scheduler: bool,
        alpha: List[int],
        gamma: int,
        reg_weight: int,
        use_kan: bool,
        ignore_Tnet: bool,
        norm_type: str,
        dropout: float
):
    if not use_kan:
        classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes, ignore_Tnet=ignore_Tnet,
                                        norm_type=norm_type, dropout=dropout).to(DEVICE)
    else:
        classifier = PointNetKAN(dim, num_points, num_classes, scaling = 2.0).to(DEVICE)
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate)
    if DEVICE == "cuda" and use_scheduler:
        scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.01, step_size_up=2000, cycle_momentum=False)
    else:
        scheduler = None
    
    trainer = PointNetTrainer(
        name=name,
        model=classifier,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion=PointNetLoss(alpha=alpha, gamma=gamma, reg_weight=reg_weight, size_average=True).to(DEVICE),
        device=DEVICE,
        train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=True),
        val_loader=DataLoader(validation_data, batch_size=batch_size, shuffle=False),
        checkpoint_dir=os.path.join(os.getcwd(), "checkpoint"),
        checkpoint_freq=checkpoint_freq
    )

    loss_dict, best_epoch, best_loss, best_acc = trainer.fit(epochs=epochs)
    #save_loss_dict(loss_dict, os.path.join(os.getcwd(), "csv", f"{name}_loss_dict.csv"))
    print(f"{name} | Best model @ epoch {best_epoch}: loss = {best_loss:.4f}, acc = {best_acc:.4f}")

# Instancias de entrenamiento
EPOCHS=200

In [6]:
# groupnorm with hard regularization: reg = 0.005 dp = 0.5
train(epochs=EPOCHS, name="gnregreg", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=None, gamma=2, reg_weight=0.005, use_kan=False, ignore_Tnet=False,
      norm_type="groupnorm", dropout=0.5)

╒═══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1   │       1.7917 │      0.2922 │     1.6181 │    0.3442 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2   │       1.6282 │      0.3355 │     1.709  │    0.3246 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3   │       1.5555 │      0.3462 │     1.4982 │    0.3604 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4   │       1.4668 │      0.3606 │     1.3376 │    0.3783 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5   │       1.4187 │      0.3654 │     1.3022 │    0.4009 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6   │       1.3308 │      0.3965 │     1.2256 │    0.4312 │
├───────────┼──────────────┼─────────────┼──────

In [7]:
# groupnorm with hard regularization and alpha inv freq
alpha = [3991/106, 3991/515, 3991/889, 3991/200, 3991/200, 3991/465, 3991/200, 3991/680, 3991/392, 3991/344]
train(epochs=EPOCHS, name="gnregregalpha", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=False, alpha=alpha, gamma=2, reg_weight=0.005, use_kan=False, ignore_Tnet=False,
      norm_type="groupnorm", dropout=0.5)

╒═══════════╤══════════════╤═════════════╤════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │   Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪════════════╪═══════════╡
│ Epoch 1   │      19.7571 │      0.1375 │    18.4927 │    0.1525 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 2   │      18.4064 │      0.1633 │    18.5712 │    0.1356 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 3   │      18.4885 │      0.1554 │    17.8914 │    0.1775 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 4   │      18.0642 │      0.1697 │    17.7966 │    0.1292 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 5   │      17.6575 │      0.182  │    16.8131 │    0.3054 │
├───────────┼──────────────┼─────────────┼────────────┼───────────┤
│ Epoch 6   │      16.8693 │      0.2316 │    16.2715 │    0.1508 │
├───────────┼──────────────┼─────────────┼──────

In [8]:
# groupnorm with hard regularization and scheduler
train(epochs=EPOCHS, name="gnregregsched", num_global_feats=num_global_feats, learning_rate=learning_rate,
      use_scheduler=True, alpha=None, gamma=2, reg_weight=0.005, use_kan=False, ignore_Tnet=False,
      norm_type="groupnorm", dropout=0.5)

╒═══════════╤══════════════╤═════════════╤══════════════════╤═══════════╕
│ Epoch     │   Train Loss │   Train Acc │         Val Loss │   Val Acc │
╞═══════════╪══════════════╪═════════════╪══════════════════╪═══════════╡
│ Epoch 1   │       2.0351 │      0.2858 │      1.7116      │    0.3358 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 2   │       1.7209 │      0.3186 │      1.5591      │    0.3025 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 3   │       1.527  │      0.3282 │      1.5136      │    0.3472 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 4   │       1.4721 │      0.3369 │      1.4442      │    0.36   │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 5   │       1.4425 │      0.3372 │      1.3896      │    0.3667 │
├───────────┼──────────────┼─────────────┼──────────────────┼───────────┤
│ Epoch 6   │       1.4798 │      0.33

# Dataset Testeo

In [9]:
# dataset de prueba
base_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[])
affine_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Rotation(), Reflection(), Scale(max_ratio=2.5)])
complex_test_data = ModelNet(classes, DatasetType.TEST, repetitions=1, preserve_original=False,
                          transformations=[Rotation(), Reflection(), Scale(max_ratio=2.5),
                                          Jittering(max_units=0.005), DropRandom(loss_ratio=0.4), Noise()])

In [11]:
def test_it(classifier_path: str, num_global_feats=num_global_feats, use_kan=False,
            ignore_Tnet=False, use_TTA=False, merge_mode=None, norm_type="batchnorm"):

    for data_name, data in [["base", base_test_data], ["affine", affine_test_data], ["complex", complex_test_data]]:
        data_loader = DataLoader(data, batch_size=batch_size, shuffle=False)
    
        if not use_kan:
            classifier = PointNetClassifier(dim, num_points, num_global_feats, num_classes,
                                            ignore_Tnet=ignore_Tnet, norm_type=norm_type).to(DEVICE)
        else:
            classifier = PointNetKAN(dim, num_points, num_classes, scaling = 1.0, ignore_Tnet=ignore_Tnet).to(DEVICE)
        
        
        classifier.load_state_dict(torch.load(classifier_path, map_location=torch.device(DEVICE)))

        if use_TTA:
            classifier = TTAClassifier(classifier=classifier, transformations=[Rotation(), Reflection()], merge_mode=merge_mode)

        
            
        with torch.no_grad():
            classifier = classifier.eval()
            correct = 0
            
            for pcds, labels in data_loader:
                pcds = pcds.to(DEVICE)
                labels = labels.squeeze().to(DEVICE)

                # Hacer predicciones
                out, _, _ = classifier(pcds)

                if not use_TTA:
                    # Calculamos las elecciones, TTA ya hace softmax.
                    out = torch.softmax(out, dim=1)
                    
                pred_choice = out.argmax(dim=1)
                
                # Elecciones correctas, acumuladas
                correct += pred_choice.eq(labels.data).cpu().sum().item()

            test_acc = correct / float(len(data))
            print(f"\tAccuracy on {data_name} dataset:\t", test_acc)

# Tests
_dir = os.path.join(os.getcwd(), "checkpoint", "best_model")
print("Groupnorm with hard regularization classifier (best epoch):")
test_it(os.path.join(_dir, "gnregreg_best_model.pth"), norm_type="groupnorm")
print("Groupnorm with hard regularization + alpha classifier (best epoch):")
test_it(os.path.join(_dir, "gnregregalpha_best_model.pth"), norm_type="groupnorm")
print("Groupnorm with hard regularization classifier + scheduler (best epoch):")
test_it(os.path.join(_dir, "gnregregsched_best_model.pth"), norm_type="groupnorm")

_dir = os.path.join(os.getcwd(), "checkpoint", "by_epoch")
print("Groupnorm with hard regularization classifier (last epoch):")
test_it(os.path.join(_dir, "gnregreg_epoch_0200.pth"), norm_type="groupnorm")
print("Groupnorm with hard regularization + alpha classifier (last epoch):")
test_it(os.path.join(_dir, "gnregregalpha_epoch_0200.pth"), norm_type="groupnorm")
print("Groupnorm with hard regularization classifier + scheduler (last epoch):")
test_it(os.path.join(_dir, "gnregregsched_epoch_0200.pth"), norm_type="groupnorm")

Groupnorm with hard regularization classifier (best epoch):
	Accuracy on base dataset:	 0.8799559471365639
	Accuracy on affine dataset:	 0.7257709251101322
	Accuracy on complex dataset:	 0.6806167400881057
Groupnorm with hard regularization + alpha classifier (best epoch):
	Accuracy on base dataset:	 0.9052863436123348
	Accuracy on affine dataset:	 0.711453744493392
	Accuracy on complex dataset:	 0.6729074889867841
Groupnorm with hard regularization classifier + scheduler (best epoch):
	Accuracy on base dataset:	 0.8865638766519823
	Accuracy on affine dataset:	 0.6651982378854625
	Accuracy on complex dataset:	 0.6112334801762115
Groupnorm with hard regularization classifier (last epoch):
	Accuracy on base dataset:	 0.8887665198237885
	Accuracy on affine dataset:	 0.7400881057268722
	Accuracy on complex dataset:	 0.6949339207048458
Groupnorm with hard regularization + alpha classifier (last epoch):
	Accuracy on base dataset:	 0.9229074889867841
	Accuracy on affine dataset:	 0.7709251101